## Nettoyage de la base allocine

In [131]:
import pandas as pd 
import numpy as np 
from datetime import datetime
from dateutil import parser 

In [116]:
df_movies= pd.read_csv("df_film_ac.csv")

In [117]:
print(df_movies.dtypes)
#df_movies.isnull().sum()
#df_movies["date"].value_counts()

Titre original           object
Note press               object
Notes spectateur         object
Critiques presse         object
Critiques spectateurs    object
id allocine               int64
Box office               object
Budget                   object
date                     object
duree                    object
dtype: object


In [118]:
#modifier les types numériques 
df_movies['Box office'] = df_movies['Box office'].str.replace(' ', '').astype(float)
df_movies['Note press'] = df_movies['Note press'].str.replace(',', '.').astype(float)
df_movies['Notes spectateur'] = df_movies['Notes spectateur'].replace("--", np.nan)
df_movies['Notes spectateur'] = df_movies['Notes spectateur'].str.replace(',', '.').astype(float)

In [119]:
# Fonction pour convertir le mois en anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)

# Remplacer les chaînes "nan" par des valeurs NaN
df_movies['date'] = df_movies['date'].replace('nan', np.nan)

# Appliquer la fonction pour convertir le mois en anglais
df_movies['date'] = df_movies['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_movies['date'] = df_movies['date'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)



In [120]:
# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Supprimer les espaces et diviser la chaîne en parties
        parts = duration_str.replace(' ', '').split('h')

        # Vérifier la présence des heures et des minutes
        if len(parts) == 2:
            hours = int(parts[0])
            minutes = 0 if 'min' not in parts[1] else int(parts[1].replace('min', ''))
            
            # Calculer la durée totale en minutes
            total_minutes = hours * 60 + minutes
            
            return int(total_minutes)
    
    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return float('nan')

# Appliquer la fonction de conversion
df_movies['duree_minutes'] = df_movies['duree'].apply(convert_duration)


In [121]:
#Extraire le nombre de critique de la presse 

def extract_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Utiliser isdigit() pour extraire uniquement les chiffres
        return np.nan if not critiques_str.split()[0].isdigit() else int(critiques_str.split()[0])
    else:
        return np.nan

# Appliquer la fonction de conversion
df_movies['Nombre_de_critiques_presse'] = df_movies['Critiques presse'].apply(extract_critiques_count)


In [122]:
#critiques et votes spectateurs 

# Fonction pour extraire le nombre de notes et le nombre de critiques
def extract_notes_and_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Trouver les nombres dans la chaîne
        numbers = [int(word) for word in critiques_str.split() if word.isdigit()]

        # Extraire le nombre de notes et de critiques en fonction de la longueur de la liste "numbers"
        if len(numbers) == 1:
            return numbers[0], np.nan
        elif len(numbers) == 2:
            return numbers[0], numbers[1]

    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return np.nan, np.nan

# Appliquer la fonction de conversion
df_movies[['Nombre_de_notes_spectateurs', 'Nombre_de_critiques_spectateurs']] = df_movies['Critiques spectateurs'].apply(extract_notes_and_critiques_count).apply(pd.Series)

In [123]:
df_movies.head()

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,31 critiques,80471 notes dont 4605 critiques,132039,127770000.0,80 000 000 $,2010-02-24,2h 17min,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,24 critiques,110095 notes dont 7212 critiques,143692,290948208.0,160 000 000 $,2010-07-21,2h 28min,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,20 critiques,52676 notes dont 2887 critiques,126693,291377000.0,150 000 000 $,2010-11-24,2h 26min,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,22 critiques,26730 notes dont 2133 critiques,126678,89981000.0,200 000 000 $,2010-05-26,2h 06min,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,20 critiques,10503 notes dont 1144 critiques,128955,92524000.0,80 000 000 $,2010-01-20,1h 49min,109.0,20.0,10503.0,1144.0


In [124]:
print(df_movies.dtypes)

Titre original                             object
Note press                                float64
Notes spectateur                          float64
Critiques presse                           object
Critiques spectateurs                      object
id allocine                                 int64
Box office                                float64
Budget                                     object
date                               datetime64[ns]
duree                                      object
duree_minutes                             float64
Nombre_de_critiques_presse                float64
Nombre_de_notes_spectateurs               float64
Nombre_de_critiques_spectateurs           float64
dtype: object


In [125]:
#on retire les colonnes que nous utilisons pas 
colonnes_a_retirer = ["Critiques presse","Critiques spectateurs","duree","Budget"]
df_movies_clean=df_movies.drop(colonnes_a_retirer,axis=1)

In [126]:
df_movies_clean.head()

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0


In [128]:
#retrait de tous les films sans critique spectateurs
#on garde que les films avec au moins plus de 5 commentaires

df_film_avec_critiques = df_movies_clean.dropna(subset=['Nombre_de_critiques_spectateurs'])
df_film_avec_critiques = df_film_avec_critiques[df_film_avec_critiques['Nombre_de_critiques_spectateurs'] >= 5]
df_film_avec_critiques


,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0
...,...,...,...,...,...,...,...,...,...,...
11101,The Gateway,NaN,2.3,273584,NaN,2022-03-16,91.0,NaN,40.0,6.0
11102,"Batman: The Long Halloween, Part Two",NaN,3.7,292641,NaN,2021-08-04,84.0,NaN,114.0,7.0
11104,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,NaN,50.0,6.0
11117,Black As Night,NaN,1.6,287632,NaN,2021-10-01,87.0,NaN,53.0,6.0


In [129]:
df_film_avec_critiques.to_csv('df_film_avec_critiques.csv', index=False)


## Nettoyage de la base TMDB

In [157]:
#engine=python permet de regler les problemes de compatibilité

df_tmdb= pd.read_csv("df_tmdb.csv", engine="python")
print(df_tmdb.dtypes)

adult                     object
backdrop_path             object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                       float64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
success                   object
status_code              float64
status_message            object
dtype: object


In [158]:
noms_variables = df_tmdb.columns.tolist()
print(noms_variables)

#for i in noms_variables:
    #print(i,":",df_tmdb[i].isnull().sum())

#on observe que les trois dernières colonnes peuvent être supprimés de la dataframe 
#trop de valeurs manquantes

colonnes_a_supprimer = ["success", "status_code", "status_message"]
df_tmdb_clean = df_tmdb.drop(columns=colonnes_a_supprimer)


['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'success', 'status_code', 'status_message']


In [ ]:
#changement format release date 
# Supposons que df_tmdb soit votre DataFrame
df_tmdb_clean['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors="coerce")
df_tmdb_clean["release_date"].value_counts()

#### on garde uniquement les films qui sont dans la base allocine avec critique 

In [154]:
ma_liste = df_film_avec_critiques['Titre original'].tolist()
len(ma_liste)

4194

## Merging des deux bases

In [168]:
#séparation du mois et de l'année de sortie dans les deux bases

# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_film_avec_critiques['mois_sortie'] = df_film_avec_critiques['date'].dt.month
df_film_avec_critiques['annee_sortie'] = df_film_avec_critiques['date'].dt.year

df_film_avec_critiques.head()

# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_tmdb_clean['mois_sortie'] = df_tmdb_clean['release_date'].dt.month
df_tmdb_clean['annee_sortie'] = df_tmdb_clean['release_date'].dt.year

df_tmdb_clean.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,mois_sortie,annee_sortie
0,False,/ecvy2kMxsJ60ej52beZ0F8EOGkL.jpg,NaN,80000000.0,"Drama, Thriller, Mystery",http://www.shutterisland.com/,11324.0,tt1130884,en,Shutter Island,...,138.0,"English, Deutsch",Released,Some places never let you go.,Shutter Island,False,8.201,22634.0,2.0,2010.0
1,False,NaN,NaN,0.0,Documentary,NaN,1157141.0,tt5445282,en,Shutter Island: Into the Lighthouse,...,21.0,English,Released,NaN,Shutter Island: Into the Lighthouse,False,0.000,0.0,6.0,2010.0
2,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,NaN,160000000.0,"Action, Science Fiction, Adventure",https://www.warnerbros.com/movies/inception,27205.0,tt1375666,en,Inception,...,148.0,"English, Français, 日本語, Kiswahili",Released,Your mind is the scene of the crime.,Inception,False,8.366,34886.0,7.0,2010.0
3,False,/JeGkRdNsOuMrgwBdtB0hp763MU.jpg,El Crack Collection,0.0,"Drama, Thriller",https://filmaxinternationalsales.com/film/the-...,613092.0,tt6793710,es,El crack cero,...,117.0,Español,Released,The prequel to the cult film El Crack,The Crack: Inception,False,6.700,36.0,10.0,2019.0
4,False,NaN,NaN,0.0,"Animation, Action, Thriller, Science Fiction",NaN,64956.0,tt5295894,en,Inception: The Cobol Job,...,14.0,English,Released,NaN,Inception: The Cobol Job,False,7.300,290.0,12.0,2010.0


In [173]:
df_film_avec_critiques['titre_normalise'] = df_film_avec_critiques['Titre original'].str.replace(r'\W', '', regex=True).str.lower()
df_tmdb_clean['titre_normalise'] = df_tmdb_clean['original_title'].str.replace(r'\W', '', regex=True).str.lower()

# Fusion sur la normalisation des titres et la date de sortie (mois et année)
df_merge = pd.merge(
    df_film_avec_critiques,
    df_tmdb_clean,
    how='inner',
    left_on=['titre_normalise'],
    right_on=['titre_normalise']
)

# Supprimer la colonne temporaire titre_normalise
df_merge.drop(columns=['titre_normalise'], inplace=True)

# Afficher le DataFrame résultant
df_merge

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,mois_sortie_y,annee_sortie_y
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0,...,138.0,"English, Deutsch",Released,Some places never let you go.,Shutter Island,False,8.201,22634.0,2.0,2010.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0,...,148.0,"English, Français, 日本語, Kiswahili",Released,Your mind is the scene of the crime.,Inception,False,8.366,34886.0,7.0,2010.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0,...,146.0,English,Released,One Way… One Fate… One Hero.,Harry Potter and the Deathly Hallows: Part 1,False,7.762,18019.0,11.0,2010.0
3,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0,...,146.0,English,Released,One Way… One Fate… One Hero.,Harry Potter and the Deathly Hallows: Part 1,False,7.762,18019.0,11.0,2010.0
4,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0,...,146.0,English,Released,One Way… One Fate… One Hero.,Harry Potter and the Deathly Hallows: Part 1,False,7.762,18019.0,11.0,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12290,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,NaN,50.0,6.0,...,80.0,English,Released,The battle to end all battles.,Mortal Kombat Legends: Battle of the Realms,False,7.686,397.0,8.0,2021.0
12291,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,NaN,50.0,6.0,...,80.0,English,Released,The battle to end all battles.,Mortal Kombat Legends: Battle of the Realms,False,7.686,397.0,8.0,2021.0
12292,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,NaN,50.0,6.0,...,80.0,English,Released,The battle to end all battles.,Mortal Kombat Legends: Battle of the Realms,False,7.686,397.0,8.0,2021.0
12293,Black As Night,NaN,1.6,287632,NaN,2021-10-01,87.0,NaN,53.0,6.0,...,87.0,English,Released,Find your own way to slay.,Black as Night,False,5.500,120.0,10.0,2021.0
